In [27]:
import csv 
import numpy as np

In [13]:
data = []
with open('meta-dataset.csv', 'r') as file:
    reader = csv.reader(file)
    header = next(reader)
    for row in reader:
        data.append(row[1:])
    data = np.array(data)
    data = data.astype(float)

In [14]:
# Calculate U_yi
y = data[:, 8]
u = [0] * 3
u[0] = (y == 0).sum() - 1
u[1] = (y == 1).sum() - 1
u[2] = (y == 2).sum() - 1

In [19]:
# Pre-calculate NN of all <xi, yi>
d = [[] for _ in range(226)] 
for i in range(len(data)):
    a = data[i][0:8]
    for j in range(i + 1, len(data)):
        b = data[j][0:8]
        val = np.sqrt(sum(a * b))
        d[i].append(val)
        d[j].append(val)

for i in range(len(d)):
    d[i].sort()


In [16]:
# Calculate b-values for all <xi, yi>
bvalues = [0.0] * 226

invd = lambda x : 1.0 / (1.0 + float(x))
invd_func = np.vectorize(invd)

for i in range(226):
    y_val = data[i][8].astype(np.uint8)
    pos = u[y_val]
    num = d[i][0:pos].copy()
    den = d[i].copy()
    num = invd_func(num)
    den = invd_func(den)
    bvalues[i] = np.sum(num) / np.sum(den)



In [29]:
# Allocate to bins
bin = np.zeros(8) 
for val in bvalues:
    num = 7 if val == 1.0 else int(val / 0.125)
    bin[num] += 1
bin = bin / 226